In [8]:
import Parameteriser
from Simulator import SolutionSimulator

rdk_pmd = Parameteriser.SolutionParameteriser.via_rdkit("c1ccccc1")
rdk_pmd


/localhome/cschiebroek/.conda/envs/mdfp_carl/lib/python3.11/site-packages/parmed/structure.py:1775: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  coords = np.array(value, dtype=np.float64, copy=False, subok=True)


<Structure 1500 atoms; 497 residues; 1004 bonds; PBC (orthogonal); parameterized>

In [1]:
import Parameteriser_constrained
from Simulator import SolutionSimulator

rdk_pmd_constrained = Parameteriser_constrained.SolutionParameteriser.via_rdkit("c1ccccc1")
rdk_pmd_constrained


/localhome/cschiebroek/.conda/envs/mdfp_carl/lib/python3.11/site-packages/parmed/structure.py:1775: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  coords = np.array(value, dtype=np.float64, copy=False, subok=True)


<Structure 1494 atoms; 495 residues; 1000 bonds; PBC (orthogonal); parameterized>

In [3]:
from Simulator import SolutionSimulator
SolutionSimulator.via_openmm(rdk_pmd_constrained, file_name = "benzene_constrained", file_path = "./", 
                             platform = "CUDA", num_steps = 5000 * 500)

AttributeError: 'NoneType' object has no attribute 'req'

In [9]:
SolutionSimulator.via_openmm(rdk_pmd, file_name = "benzene", file_path = "./", 
                             platform = "CUDA", num_steps = 5000 * 500)

'/localhome/cschiebroek/MDFPs/mdfptools-1/examples/benzene.h5'

In [10]:
#First load in the simulated trajectory
import mdtraj as md
traj = md.load("./benzene.h5")

/localhome/cschiebroek/.conda/envs/mdfp_carl/lib/python3.11/site-packages/mdtraj/core/trajectory.py:439: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')


In [15]:
from Composer import SolutionComposer
mdfp = SolutionComposer.run(traj, rdk_pmd,smiles="c1ccccc1")

The returned object from a Composer is a `MDFP` object. As can be seen from above, it contains more information.
To use it for the subsequent machine learning tasks, call the `get_mdfp()` method to get the feature vectors (i.e. just the values not the keys)

In [27]:
print(mdfp.get_mdfp())

[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8.449853464132135, 0.08582111145660468, 8.441295681995182, 22.377345228101785, 0.554998685766172, 22.257757119441322, -16.95474881151296, 9.940047292255558, -16.481335399377038, -15.582322274073302, 5.718904951407143, -16.232199446655756, 30.827198692233917, 0.5674452115357036, 30.698609726670785, -32.53707108558626, 9.473301252108618, -32.46123091060353, 0.15048547845062343, 7.22533132975676e-05, 0.1505005645108472, 2.4337163, 0.008083881, 2.4337869]


In [29]:
from openff.toolkit.utils import get_data_file_path
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import numpy as np
from simtk import unit
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler
import parmed

sdf_filepath = get_data_file_path('systems/monomers/water.sdf')
molecule = Molecule(sdf_filepath)
molecule.partial_charges = np.asarray([-0.7850000262260437,0.39250001311302185,0.39250001311302185]) * unit.elementary_charge
library_charge_type = LibraryChargeHandler.LibraryChargeType.from_molecule(molecule)
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield["LibraryCharges"].add_parameter(parameter=library_charge_type)
topology = molecule.to_topology()
openmm_system = forcefield.create_openmm_system(topology, charge_from_molecules=[molecule])
structure =  parmed.openmm.topsystem.load_topology(topology.to_openmm(), openmm_system, [[1,0,0],[0,1,0],[0,0,1]])
structure.save("tip3p_updated_carl.prmtop")

AttributeError: 'NoneType' object has no attribute 'used'

In [35]:
smiles = 'c1ccccc1'
rdk_pmd = Parameteriser.SolutionParameteriser.via_rdkit(smiles)
rdk_pmd.
# ch = forcefield.get_parameter_handler('Constraints')
# ch.add_parameter({'smirks': '[#1:1]-[*:2]'})
# print(ch)

/localhome/cschiebroek/.conda/envs/mdfp_carl/lib/python3.11/site-packages/parmed/structure.py:1775: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  coords = np.array(value, dtype=np.float64, copy=False, subok=True)
